# Installations and Setup

In [1]:
from google.colab import drive
import sys

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# TODO: Mount your Google Drive; this allows the runtime environment to access your drive.
drive.mount('/content/gdrive')

In [1]:
import IPython

ipython = IPython.get_ipython()
ipython.run_line_magic("load_ext", "autoreload")
ipython.run_line_magic("autoreload", "2")

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import lightning as L
from lightning.pytorch import loggers as pl_loggers

random.seed(0)
torch.manual_seed(0)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/jeffreyxiang/opt/anaconda3/envs/4782/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/jeffreyxiang/opt/anaconda3/envs/4782/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/jeffreyxiang/opt/anaconda3/envs/4782/lib/python3.11/site-packages/ipykernel/kernelapp.py", 

In [3]:
from models.architecture.assembled import AssembledModel
from models.train_loop import PatchTSTTrainer
from data.data_loader import TSDataLoader

In [ ]:
seq_len = 24
patch_tst = AssembledModel(seq_len=seq_len)

# Train Model

In [ ]:
output_dir = "./models/results"
logs_output_dir = "./models/results/logs"
model_trainer = PatchTSTTrainer(patch_tst, output_dir)
device = "cpu"
dataloader = TSDataLoader("./data/data_files/electricity/electricity.csv", seq_len=seq_len, device=device)
train_dataloader, val_dataloader = dataloader.get_data_loaders()

tb_logger = pl_loggers.TensorBoardLogger(save_dir=logs_output_dir)

trainer = L.Trainer(max_epochs=20, 
                    check_val_every_n_epoch=2,
                    num_sanity_val_steps=0, 
                    logger=tb_logger
                    )

/Users/jeffreyxiang/Desktop/HW/cs4782_s25_final_project/data/data_loader.py:30: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_stamp['hour'] = df_stamp.date.apply(lambda row: row.hour, 1)
/Users/jeffreyxiang/Desktop/HW/cs4782_s25_final_project/data/data_loader.py:31: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_stamp['day'] = df_stamp.date.apply(lambda row: row.day, 1)
/Users/jeffreyxiang/Desktop/HW/cs4782_s25_final_project/data/data_loader.py:32: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_stamp['month'] = df_stamp.date.apply(lambda row: row.month, 1)
/Users/jeffreyxiang/Desktop/

# Evaluate Model

In [6]:
print("Starting training...")
trainer.fit(model_trainer, train_dataloader, val_dataloader)
print("Training completed.")


  | Name  | Type           | Params | Mode 
-------------------------------------------------
0 | model | AssembledModel | 42.6 M | train
1 | loss  | MSELoss        | 0      | train
-------------------------------------------------
42.6 M    Trainable params
0         Non-trainable params
42.6 M    Total params
170.290   Total estimated model params size (MB)
168       Modules in train mode
0         Modules in eval mode
/Users/jeffreyxiang/opt/anaconda3/envs/4782/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Starting training...
Epoch 0:   0%|          | 0/656 [00:00<?, ?it/s] x shape after normalization: torch.Size([32, 10, 24])
3 torch.Size([32, 10, 3, 16])
patches shape after patching: torch.Size([32, 10, 3, 16])
x shape after variable splitting: 320
x[0] shape after variable splitting: torch.Size([3, 16])
x shape before projection: 320
embeddings shape after projection: torch.Size([320, 3, 768])
pos_embeddings shape after positional encoding: torch.Size([320, 3, 768])
encoded shape after encoding: torch.Size([320, 3, 768])
encoded shape after encoding: torch.Size([320, 3, 768])
Mean shape after encoding: torch.Size([32, 10, 1])
Std shape after encoding: torch.Size([32, 10, 1])


RuntimeError: The size of tensor a (3) must match the size of tensor b (10) at non-singleton dimension 1